In [29]:
import pandas as pd
import sqlite3

In [30]:
con=sqlite3.connect('library_system.sqlite')

In [40]:
all_tables=pd.read_sql('''select * from sqlite_master''',con)
all_tables

,type,name,tbl_name,rootpage,sql
0,table,Authors,Authors,2,CREATE TABLE Authors (\n author_id INTEGER ...
1,table,Books,Books,3,CREATE TABLE Books (\n book_id INTEGER PRIM...
2,table,Members,Members,4,CREATE TABLE Members (\n member_id INTEGER ...
3,table,Borrowings,Borrowings,5,CREATE TABLE Borrowings (\n borrowing_id IN...
4,table,Fines,Fines,6,CREATE TABLE Fines (\n fine_id INTEGER PRIM...
5,view,OverdueBooks,OverdueBooks,0,CREATE VIEW OverdueBooks AS\nSELECT br.borrowi...


In [32]:
df=pd.read_sql('''select * from Authors''',con)
df

,author_id,name,country
0,1,Haruki Murakami,Japan
1,2,George Orwell,United Kingdom
2,3,Jane Austen,United Kingdom
3,4,Mark Twain,United States
4,5,Chinua Achebe,Nigeria


In [33]:
#Top 5 Most Borrowed Books
most_borrowed_books=pd.read_sql('''SELECT b.title, COUNT(*) AS borrow_count
FROM Borrowings br
JOIN Books b ON br.book_id = b.book_id
GROUP BY b.title
ORDER BY borrow_count DESC
LIMIT 5;
''',con)
most_borrowed_books

,title,borrow_count
0,Pride and Prejudice,2
1,Norwegian Wood,2
2,Kafka on the Shore,2
3,1984,2
4,Things Fall Apart,1


In [34]:
#Members With the Highest Total Fines
highest_fines=pd.read_sql('''SELECT m.name, SUM(f.amount) AS total_fines
FROM Fines f
JOIN Borrowings br ON f.borrowing_id = br.borrowing_id
JOIN Members m ON br.member_id = m.member_id
GROUP BY m.name
ORDER BY total_fines DESC;
''',con)
highest_fines

,name,total_fines
0,Charlie Lee,5.0
1,Evan Green,4.5
2,Alice Smith,1.5
3,Bob Johnson,0.5
4,Dana White,0.0


In [43]:
#Most Active Members by Number of Borrowings
most_active_members=pd.read_sql('''SELECT m.name, COUNT(*) AS borrow_count
FROM Borrowings br
JOIN Members m ON br.member_id = m.member_id
GROUP BY m.name
ORDER BY borrow_count DESC
LIMIT 5;
''',con)
most_active_members

,name,borrow_count
0,Evan Green,2
1,Dana White,2
2,Charlie Lee,2
3,Bob Johnson,2
4,Alice Smith,2


In [36]:
#Most Active Members Per Month
most_active_members_per_month=pd.read_sql('''SELECT 
    strftime('%Y-%m', br.borrow_date) AS month,
    m.name,
    COUNT(*) AS borrow_count
FROM Borrowings br
JOIN Members m ON br.member_id = m.member_id
GROUP BY month, m.name
ORDER BY month, borrow_count DESC;
''',con)
most_active_members_per_month

,month,name,borrow_count
0,2023-01,Bob Johnson,1
1,2023-01,Alice Smith,1
2,2023-02,Alice Smith,1
3,2023-03,Dana White,1
4,2023-03,Charlie Lee,1
5,2023-04,Evan Green,1
6,2023-04,Dana White,1
7,2023-04,Charlie Lee,1
8,2023-04,Bob Johnson,1
9,2023-05,Evan Green,1


In [37]:
#Rank Books by Popularity Using a Window Function
using_window_function=pd.read_sql('''SELECT title, borrow_count,
       RANK() OVER (ORDER BY borrow_count DESC) AS rank
FROM (
    SELECT b.title, COUNT(*) AS borrow_count
    FROM Borrowings br
    JOIN Books b ON br.book_id = b.book_id
    GROUP BY b.title
);
''',con)
using_window_function

,title,borrow_count,rank
0,1984,2,1
1,Kafka on the Shore,2,1
2,Norwegian Wood,2,1
3,Pride and Prejudice,2,1
4,Adventures of Huckleberry Finn,1,5
5,Things Fall Apart,1,5


In [41]:
# Overdue Books
overdue_books=pd.read_sql('''SELECT br.borrowing_id, b.title, m.name, br.borrow_date
FROM Borrowings br
JOIN Books b ON br.book_id = b.book_id
JOIN Members m ON br.member_id = m.member_id
WHERE br.return_date IS NULL AND DATE('now') > DATE(br.borrow_date, '+14 day')
''',con)
overdue_books

,borrowing_id,title,name,borrow_date
0,4,Pride and Prejudice,Charlie Lee,2023-03-01
1,8,Norwegian Wood,Charlie Lee,2023-04-20
2,10,Pride and Prejudice,Evan Green,2023-05-01


In [42]:
# Overdue Fine Calculation
overdue_fine_collection=pd.read_sql('''WITH OverdueDays AS (
    SELECT 
        br.borrowing_id,
        julianday(CASE 
                    WHEN br.return_date IS NOT NULL THEN br.return_date
                    ELSE DATE('now')
                 END) - julianday(br.borrow_date) - 14 AS overdue_days
    FROM Borrowings br
    WHERE 
        (br.return_date IS NULL AND DATE('now') > DATE(br.borrow_date, '+14 day')) OR
        (br.return_date IS NOT NULL AND DATE(br.return_date) > DATE(br.borrow_date, '+14 day'))
),
CalculatedFines AS (
    SELECT 
        borrowing_id,
        MAX(0, ROUND(overdue_days)) * 1.0 AS calculated_fine
    FROM OverdueDays
)
SELECT * FROM CalculatedFines;
''',con)
overdue_fine_collection

,borrowing_id,calculated_fine
0,4,775.0
1,8,725.0
2,10,714.0
